In [1]:
########### 1) LIBRERIAS A UTILIZAR ################# 
source("./ins_paquetes.R")
ins_paquetes('magrittr', 'xgboost')

Warning message:
"package 'xgboost' was built under R version 3.6.3"


Bellow Packages Successfully Installed:

magrittr  xgboost 
    TRUE     TRUE 


In [2]:
library(rattle)   # Access the weather dataset and utilities.
library(magrittr) # Utilise %>% and %<>% pipeline operators.
library(xgboost)

building <- TRUE
scoring  <- ! building

Warning message:
"package 'rattle' was built under R version 3.6.3"Loading required package: tibble
Warning message:
"package 'tibble' was built under R version 3.6.3"Loading required package: bitops
Rattle: A free graphical interface for data science with R.
Versión 5.4.0 Copyright (c) 2006-2020 Togaware Pty Ltd.
Escriba 'rattle()' para agitar, sacudir y  rotar sus datos.

Attaching package: 'rattle'

The following object is masked from 'package:xgboost':

    xgboost



In [20]:
# A pre-defined value is used to reset the random seed 
# so that results are repeatable.

crv$seed <- 42 

In [21]:
#=======================================================================

# Load a dataset from file.

library(readxl, quietly=TRUE)

In [22]:
crs$dataset <- read_excel("./datasets/Marvel_Travel.xlsx", guess_max=1e4)
#dataset <- read_excel("D:/CIS/2020-2B/PARTICIPACIONES EN CLASE/P6_GRUPAL/Marvel_Travel.xlsx", guess_max=1e4)

head(crs$dataset)

Referencia_ID,CUM_Vta,Prod_B,Prod_C,Prod_D,Prod_E,APLICA
ID_CLIENTE_4871,13,56,14,29,7,0
ID_CLIENTE_5187,13,81,17,48,5,1
ID_CLIENTE_2592,13,161,61,114,25,1
ID_CLIENTE_2678,13,29,2,14,1,1
ID_CLIENTE_3118,13,85,31,48,7,1
ID_CLIENTE_2819,13,114,17,45,6,1


In [23]:
str(crs$dataset)

tibble [3,087 x 7] (S3: tbl_df/tbl/data.frame)
 $ Referencia_ID: chr [1:3087] "ID_CLIENTE_4871" "ID_CLIENTE_5187" "ID_CLIENTE_2592" "ID_CLIENTE_2678" ...
 $ CUM_Vta      : num [1:3087] 13 13 13 13 13 13 13 13 13 13 ...
 $ Prod_B       : num [1:3087] 56 81 161 29 85 114 61 22 33 31 ...
 $ Prod_C       : num [1:3087] 14 17 61 2 31 17 9 5 13 10 ...
 $ Prod_D       : num [1:3087] 29 48 114 14 48 45 44 8 16 10 ...
 $ Prod_E       : num [1:3087] 7 5 25 1 7 6 4 2 3 4 ...
 $ APLICA       : num [1:3087] 0 1 1 1 1 1 1 1 0 0 ...


In [24]:
#=======================================================================
#setdiff (MINUS or EXCEPT in SQL) 
#fsetdiff will return max(0, xn-yn) copies of that row.
#Generate regular sequences. seq is a standard generic with a default
#method. seq_len are very fast

set.seed(crv$seed)
crs$nobs <- nrow(crs$dataset)
crs$train <- sample(crs$nobs, 0.7*crs$nobs)

In [25]:
crs$nobs %>%
  seq_len() %>%
  setdiff(crs$train) %>%
  sample(0.15*crs$nobs) ->
  crs$validate

In [26]:
#?seq_len()

crs$nobs %>%
  seq_len() %>%
  setdiff(crs$train) %>%
  setdiff(crs$validate) ->
  crs$test

In [27]:
# Se han anotado las siguientes selecciones de variable.

crs$input     <- c("CUM_Vta", "Prod_B", "Prod_C", "Prod_D",
                   "Prod_E")

crs$numeric   <- c("CUM_Vta", "Prod_B", "Prod_C", "Prod_D",
                   "Prod_E")

crs$categoric <- NULL

crs$target    <- "APLICA"
crs$risk      <- NULL
crs$ident     <- NULL
crs$ignore    <- "Referencia_ID"
crs$weights   <- NULL

In [28]:
#=======================================================================
# Extreme Boost 

# The `xgboost' package implements the extreme gradient boost algorithm.
# Construir el modelo Extreme Boost.

#crs$dataset[crs$train,c(crs$input, crs$target)]

set.seed(crv$seed)

crs$ada <- xgboost(APLICA ~ .,
                   data              = crs$dataset[crs$train,c(crs$input, crs$target)],
                   max_depth         = 6,
                   eta               = 0.3, 
                   num_parallel_tree = 1, 
                   nthread           = 2, 
                   nround            = 50,
                   metrics           = 'error',
                   objective         = 'binary:logistic')

Warning message in xgb.get.DMatrix(data, label, missing, weight):
"xgboost: label will be ignored."

ERROR: Error in xgb.get.DMatrix(data, label, missing, weight): xgboost doesn't support data.frame as input. Convert it to matrix first.


In [1]:
# Imprimir los resultados del modelo.

print(crs$ada)

cat('\nFinal iteration error rate:\n')
print(round(crs$ada$evaluation_log[crs$ada$niter, ], 2))

cat('\nImportance/Frequency of variables actually used:\n')
print(crs$imp <- importance(crs$ada, crs$dataset[crs$train,c(crs$input, crs$target)]))

# Tiempo transcurrido: 0.12 segs

# Diagramar la importancia relativa de las variables.

ggVarImp(crs$ada)

# Plot the error rate as we increase the number of iteration.

plot(crs$ada$evaluation_log, type='o')

# Evaluate model performance on the testing dataset. 

# Curva ROC: necesita el paquete ROCR.

library(ROCR)

# ROC Curve: requires the ggplot2 package.

library(ggplot2, quietly=TRUE)

# Generar una curva ROC para el modelo xgb en Marvel_Travel.xlsx [prueba].

crs$pr <- predict(crs$ada, crs$dataset[crs$test, c(crs$input, crs$target)])
crs$pr



# Remove observations with missing target.
no.miss   <- na.omit(crs$dataset[crs$test, c(crs$input, crs$target)]$APLICA)
no.miss

pred <- prediction(crs$pr, no.miss)


pe <- performance(pred, "tpr", "fpr")
au <- performance(pred, "auc")@y.values[[1]]
pd <- data.frame(fpr=unlist(pe@x.values), tpr=unlist(pe@y.values))
p <- ggplot(pd, aes(x=fpr, y=tpr))
p <- p + geom_line(colour="red")
p <- p + xlab("False Positive Rate") + ylab("True Positive Rate")
p <- p + ggtitle("ROC Curve Extreme Boost Marvel_Travel.xlsx [prueba] APLICA")
p <- p + theme(plot.title=element_text(size=10))
p <- p + geom_line(data=data.frame(), aes(x=c(0,1), y=c(0,1)), colour="grey")
p <- p + annotate("text", x=0.50, y=0.00, hjust=0, vjust=0, size=5,
                  label=paste("AUC =", round(au, 2)))
print(p)

# Calcular el área bajo la curva del diagrama.


# Evaluate model performance on the testing dataset. 

# Generar una matriz de error para el modelo Extreme Boost.

# Obtenga la respuesta del modelo Extreme Boost.

lvls <- levels(as.factor(crs$dataset[[crs$target]]))
lvls[2]


crs$pr <- factor(ifelse(predict(crs$ada, crs$dataset[crs$test, c(crs$input, crs$target)]) > 0.5,
                        lvls[2], lvls[1]))
crs$pr
# Generate the confusion matrix showing counts.

rattle::errorMatrix(crs$dataset[crs$test, c(crs$input, crs$target)]$APLICA, crs$pr, count=TRUE)

###########################################################
#errorMatrix(actual,
#            predicted,
#            percentage=TRUE,
#            digits=ifelse(percentage,1,3),
#            count=FALSE)
#Arguments
#actual	: a vector of true values.
#predicted:a vector of predicted values.
#percentage	:return percentages.
#digits	:the number of digits to round results.
#count:return counts.
############################################################

# Generate the confusion matrix showing proportions.

per <- rattle::errorMatrix(crs$dataset[crs$test, c(crs$input, crs$target)]$APLICA, crs$pr)

# Calculate the overall error percentage.

cat(100-sum(diag(per), na.rm=TRUE))

ERROR: Error in crv$seed <- 42: objeto 'crv' no encontrado
